In [51]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
DATASET_PATH = "C:\\Users\\PC\\Desktop\\spam_dataset"
SPAM_PATH = os.path.join(DATASET_PATH , "spam")
HAM_PATH = os.path.join(DATASET_PATH , "easy_ham")


        

In [52]:
spam_emails = [ email for email in os.listdir(SPAM_PATH) if len(email) > 20 ]
ham_emails = [ email for email in os.listdir(HAM_PATH) if len(email) > 20 ]

In [53]:
print("spam mails :" , len(spam_emails))
print("ham mails :" , len(ham_emails))

spam mails : 500
ham mails : 2551


In [54]:
import email
import email.policy

def load_email(filename , path):
    
    with open(os.path.join(path,filename) , "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)
    

In [55]:
spam_emails = [ load_email(email , SPAM_PATH) for email in spam_emails ]
ham_emails = [ load_email(email , HAM_PATH) for email in ham_emails ]

In [56]:
print(spam_emails[16].get_content().strip())

Help wanted.  We are a 14 year old fortune 500 company, that is
growing at a tremendous rate.  We are looking for individuals who
want to work from home.

This is an opportunity to make an excellent income.  No experience
is required.  We will train you.

So if you are looking to be employed from home with a career that has
vast opportunities, then go:

http://www.basetel.com/wealthnow

We are looking for energetic and self motivated people.  If that is you
than click on the link and fill out the form, and one of our
employement specialist will contact you.

To be removed from our link simple go to:

http://www.basetel.com/remove.html


1349lmrd5-948HyhJ3622xXiM0-290VZdq6044fFvN0-799hUsU07l50


In [57]:
def get_email_structure(email):
    return email.get_content_type()

In [58]:
from collections import Counter

def email_structure(emails):
    hashTable = Counter()
    for email in emails:
        structure = get_email_structure(email)
        hashTable[structure] += 1
    return hashTable

print(email_structure(spam_emails).most_common(),"\n\n")
print(email_structure(ham_emails).most_common())

[('text/plain', 218), ('text/html', 183), ('multipart/alternative', 47), ('multipart/mixed', 43), ('multipart/related', 9)] 


[('text/plain', 2453), ('multipart/signed', 74), ('multipart/mixed', 10), ('multipart/alternative', 9), ('multipart/related', 3), ('multipart/report', 2)]


In [59]:
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(spam_emails + ham_emails , dtype=object)
y = np.array([1] * len(spam_emails) + [0] *len(ham_emails))

X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=42 , test_size=0.2)


In [60]:

webpage = spam_emails[0].get_content()
pageText = BeautifulSoup(webpage,"html.parser")
pageText.get_text().replace("\n","").replace("    "," ").strip()



"Save up to 70% on Life Insurance.Why Spend More Than You Have To?Life Quote SavingsEnsuring your    family's financial security is very important. Life Quote Savings makes    buying life insurance simple and affordable. We Provide FREE Access to The    Very Best Companies and The Lowest Rates.Life Quote Savings is FAST, EASY and    SAVES you money! Let us help you get started with the best values in    the country on new coverage. You can SAVE hundreds or even thousands    of dollars by requesting a FREE quote from Lifequote Savings. Our    service will take you less than 5 minutes to complete. Shop and    compare. SAVE up to 70% on all types of Life insurance! Click Here For Your    Free Quote!Protecting your family is the best investment you'll ever     make!If you are in receipt of this email    in error and/or wish to be removed from our list, PLEASE CLICK HERE AND TYPE REMOVE. If you    reside in any state which prohibits e-mail solicitations for insurance,    please disregard th

In [61]:
from bs4 import BeautifulSoup

def html_to_plain_text(html):
    htmlText = BeautifulSoup(html,"html.parser")
    return htmlText.get_text().strip()

def email_to_text(email):
    
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except: # in case of encoding issues
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content
    if html:
        return html_to_plain_text(html)
        

In [62]:
html_spam_emails = [mail for mail in X_train[y_train == 1] if get_email_structure(mail) == 'text/html' ]
sample_html_spam = html_spam_emails[1]
sample_html_spam.get_content()

'<html>\n\n<head>\n<meta http-equiv="Content-Type"\ncontent="text/html; charset=iso-8859-1">\n<meta name="GENERATOR" content="Microsoft FrontPage 5.0">\n<title></title>\n</head>\n\n<body bgcolor="#FFFFFF">\n<div align="center"><center>\n\n<table border="0" cellpadding="5" width="90%">\n    <tr>\n        <td width="100%"><hr>\n        <p align="center"><font color="#FF0000" size="5"\n        face="Arial"><strong>EBAY AUCTION NEWS</strong><br>\n        </font><font color="#0000FF" face="Arial"><strong>**\n        Recommended Resource - Special Edition **</strong><br>\n        </font><font face="Arial"><strong>Monday, August 26th, 2002</strong></font></p>\n        <hr>\n        <h3 align="center"><font color="#FF0000" face="Arial">Multiple\n        Streams of revenue using eBay and Internet: </font></h3>\n        <p align="center"><font color="#0000FF" size="4"\n        face="Arial"><strong>FREE Auction Profits Toolkit and\n        FREE training class</strong><br>\n        </font><font fa

In [63]:
email_to_text(sample_html_spam)

'EBAY AUCTION NEWS\n**\n        Recommended Resource - Special Edition **\nMonday, August 26th, 2002\n\nMultiple\n        Streams of revenue using eBay and Internet: \nFREE Auction Profits Toolkit and\n        FREE training class\n(For the first 200 respondents)\nYou have\n        been selected to participate in this FREE offer. \n\n        This eBay and Internet e-course, live web training\n        conference, and Auction Profit Toolkit could easily sell\n        for $297 - but it\'s yours absolutely FREE!\n        \n\n        This special FREE offer has been brought\n        to you by your friends at eBay Auction News. Craig Meyer\n        (The Auction Man) has agreed to provide you a free\n        "live" training class that you won\'t\n        want to miss.\n\n        You might say "Craig Meyer - the Auction Man?"\n        That\'s because most of you remember Craig as the real estate\n        guru who in 1997 helped over 2,000 families buy their own\n        homes using little or no

In [64]:
import nltk
stemmer = nltk.PorterStemmer()

In [65]:
pip install urlextract

Note: you may need to restart the kernel to use updated packages.


In [66]:
import urlextract 

url_extractor = urlextract.URLExtract()
url_extractor.find_urls("simmy simy ay simmy ay simmy yaaaa github.com")


['github.com']

In [83]:
from sklearn.base import BaseEstimator , TransformerMixin
import re

class EmailToWordCounterTransformer(BaseEstimator , TransformerMixin):
    
    def __init__(self , strip_headers = True , lower_case = True , 
                 remove_punctuation = True , replace_url = True , replace_numbers  = True , stemming = True):
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_url = replace_url
        self.replace_numbers = replace_numbers
        self.stemming = stemming 
        
    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        X_transformed = []
        
        for email in X:
            text = email_to_text(email) or ""
            
            if self.lower_case:
                text = text.lower()
            if self.replace_url:
                urls = list(set(url_extractor.find_urls(text)))
                for url in urls:
                    text = text.replace(url , " URL ")
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', 'NUMBER', text)
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text, flags=re.M)
                
            word_count = Counter(text.split())
            
            if self.stemming:
                stemmed_word_count = Counter()
                
                for word , count in word_count.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_count[stemmed_word] += count
                    
                word_count = stemmed_word_count
                X_transformed.append(word_count)
                
        return np.array(X_transformed)   


In [94]:
some_data = X_train[:10]
some_data_prepared = EmailToWordCounterTransformer().fit_transform(some_data)
some_data_prepared

array([Counter({'number': 62, 'the': 20, 'to': 14, 'ge': 10, 'compani': 10, 'lifetim': 8, 'product': 8, 'for': 8, 'and': 7, 'premium': 7, 'of': 7, 'polici': 6, 'by': 5, 'life': 5, 'insur': 5, 'is': 4, 'age': 4, 'protectorsm': 4, 'in': 4, 'from': 4, 'thi': 4, 'url': 4, 'protector': 3, 'a': 3, 'capit': 3, 'assur': 3, 'first': 3, 'coloni': 3, 'not': 3, 'underwritten': 2, 'gener': 2, 'electr': 2, 'competit': 2, 'face': 2, 'annual': 2, 'csv': 2, 's': 2, 't': 2, 'financi': 2, 'subject': 2, 'condit': 2, 'et': 2, 'al': 2, 'all': 2, 'state': 2, 'numberpremium': 2, 'guarante': 2, 'provis': 2, 'refer': 2, 'design': 2, 'requir': 2, 'as': 2, 'valu': 2, 'current': 2, 'each': 2, 'column': 2, 'here': 2, 'receiv': 2, 'mail': 2, 'profession': 2, 'ani': 1, 'way': 1, 'you': 1, 'slice': 1, 'it': 1, 'cut': 1, 'abov': 1, 'univers': 1, 'cook': 1, 'with': 1, 'male': 1, 'best': 1, 'amount': 1, 'premiumnumb': 1, 'amt': 1, 'j': 1, 'i': 1, 'l': 1, 'u': 1, 'm': 1, 'g': 1, 'sourc': 1, 'industri': 1, 'market': 1, 're

In [117]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [123]:
some_data_prepared_vocab = WordCounterToVectorTransformer().fit_transform(some_data_prepared)
some_data_prepared_vocab[0].toarray()

array([[ 0, 62, 20, ...,  0,  0,  0]], dtype=int32)

In [126]:
from sklearn.pipeline import Pipeline

full_pipeline = Pipeline([
    ("email_to_word_counter" , EmailToWordCounterTransformer()),
    ("word_counter_to_vector" , WordCounterToVectorTransformer())
])

X_train_prepared = full_pipeline.fit_transform(X_train)


In [128]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(solver="lbfgs" , max_iter=1000 , random_state=42)
scores = cross_val_score(log_clf , X_train_prepared , y_train , cv=3)
scores.mean()

0.9831999057091307

In [135]:
from sklearn.metrics import precision_score , recall_score , f1_score

X_test_prepared = full_pipeline.transform(X_test)

log_clf = LogisticRegression(solver="lbfgs", max_iter=1000, random_state=42)
log_clf.fit(X_train_prepared, y_train)

y_test_pred = log_clf.predict(X_test_prepared)

precision_score_val = precision_score(y_test,y_test_pred)
recall_score_val = recall_score(y_test,y_test_pred)
f1_score_val = f1_score(y_test,y_test_pred)

print("precision : " , precision_score_val)
print("recall :" , recall_score_val)
print("f1 score :" , f1_score_val)

precision :  1.0
recall : 0.9298245614035088
f1 score : 0.9636363636363636
